In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import Ridge, RidgeCV, LinearRegression
from sklearn.model_selection import cross_val_score, train_test_split, KFold
from sklearn.metrics import mean_squared_error,r2_score, mean_absolute_error
from sklearn.preprocessing import LabelEncoder, StandardScaler
import warnings

np.random.seed(2)
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv('Data/kc_final.csv', index_col = 0)

In [3]:
df

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,2014-10-13,221900.0,3,1.00,1180,5650,1.0,0,0,...,7,1180,0,1955,0,98178,47.5112,-122.257,1340,5650
1,6414100192,2014-12-09,538000.0,3,2.25,2570,7242,2.0,0,0,...,7,2170,400,1951,1991,98125,47.7210,-122.319,1690,7639
2,5631500400,2015-02-25,180000.0,2,1.00,770,10000,1.0,0,0,...,6,770,0,1933,0,98028,47.7379,-122.233,2720,8062
3,2487200875,2014-12-09,604000.0,4,3.00,1960,5000,1.0,0,0,...,7,1050,910,1965,0,98136,47.5208,-122.393,1360,5000
4,1954400510,2015-02-18,510000.0,3,2.00,1680,8080,1.0,0,0,...,8,1680,0,1987,0,98074,47.6168,-122.045,1800,7503
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21608,263000018,2014-05-21,360000.0,3,2.50,1530,1131,3.0,0,0,...,8,1530,0,2009,0,98103,47.6993,-122.346,1530,1509
21609,6600060120,2015-02-23,400000.0,4,2.50,2310,5813,2.0,0,0,...,8,2310,0,2014,0,98146,47.5107,-122.362,1830,7200
21610,1523300141,2014-06-23,402101.0,2,0.75,1020,1350,2.0,0,0,...,7,1020,0,2009,0,98144,47.5944,-122.299,1020,2007
21611,291310100,2015-01-16,400000.0,3,2.50,1600,2388,2.0,0,0,...,8,1600,0,2004,0,98027,47.5345,-122.069,1410,1287


# Preprocessing

In [4]:
df = df.drop_duplicates()
df.shape

(21613, 21)

In [5]:
df.id.nunique()

21436

In [6]:
del df['id']

In [7]:
df.isnull().sum()

date             0
price            0
bedrooms         0
bathrooms        0
sqft_living      0
sqft_lot         0
floors           0
waterfront       0
view             0
condition        0
grade            0
sqft_above       0
sqft_basement    0
yr_built         0
yr_renovated     0
zipcode          0
lat              0
long             0
sqft_living15    0
sqft_lot15       0
dtype: int64

In [8]:
# df['yr_renovated'] = [df.loc[i,'yr_built'] if(df.loc[i,'yr_renovated'] == 0) else df.loc[i,'yr_renovated'] for i in range(len(df)) ]
# df

In [9]:
labelencoder = LabelEncoder()
df['date'] = labelencoder.fit_transform(df['date'])
df

,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,164,221900.0,3,1.00,1180,5650,1.0,0,0,3,7,1180,0,1955,0,98178,47.5112,-122.257,1340,5650
1,220,538000.0,3,2.25,2570,7242,2.0,0,0,3,7,2170,400,1951,1991,98125,47.7210,-122.319,1690,7639
2,290,180000.0,2,1.00,770,10000,1.0,0,0,3,6,770,0,1933,0,98028,47.7379,-122.233,2720,8062
3,220,604000.0,4,3.00,1960,5000,1.0,0,0,5,7,1050,910,1965,0,98136,47.5208,-122.393,1360,5000
4,283,510000.0,3,2.00,1680,8080,1.0,0,0,3,8,1680,0,1987,0,98074,47.6168,-122.045,1800,7503
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21608,19,360000.0,3,2.50,1530,1131,3.0,0,0,3,8,1530,0,2009,0,98103,47.6993,-122.346,1530,1509
21609,288,400000.0,4,2.50,2310,5813,2.0,0,0,3,8,2310,0,2014,0,98146,47.5107,-122.362,1830,7200
21610,52,402101.0,2,0.75,1020,1350,2.0,0,0,3,7,1020,0,2009,0,98144,47.5944,-122.299,1020,2007
21611,252,400000.0,3,2.50,1600,2388,2.0,0,0,3,8,1600,0,2004,0,98027,47.5345,-122.069,1410,1287


# Correlation matrix

In [10]:
corr = df.loc[:, df.columns != 'price'].corr()
corr.style.background_gradient(cmap='coolwarm')

,date,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
date,1.000000,-0.016964,-0.034481,-0.034570,0.006361,-0.022395,0.001544,-0.001837,-0.050755,-0.040040,-0.027890,-0.019554,-0.000471,-0.024401,0.001376,-0.032851,-0.007205,-0.031653,0.002643
bedrooms,-0.016964,1.000000,0.515884,0.576671,0.031703,0.175429,-0.006582,0.079532,0.028472,0.356967,0.477600,0.303093,0.154178,0.018841,-0.152668,-0.008931,0.129473,0.391638,0.029244
bathrooms,-0.034481,0.515884,1.000000,0.754665,0.087740,0.500653,0.063744,0.187737,-0.124982,0.664983,0.685342,0.283770,0.506019,0.050739,-0.203866,0.024573,0.223042,0.568634,0.087175
sqft_living,-0.034570,0.576671,0.754665,1.000000,0.172826,0.353949,0.103818,0.284611,-0.058753,0.762704,0.876597,0.435043,0.318049,0.055363,-0.199430,0.052529,0.240223,0.756420,0.183286
sqft_lot,0.006361,0.031703,0.087740,0.172826,1.000000,-0.005201,0.021604,0.074710,-0.008958,0.113621,0.183512,0.015286,0.053080,0.007644,-0.129574,-0.085683,0.229521,0.144608,0.718557
floors,-0.022395,0.175429,0.500653,0.353949,-0.005201,1.000000,0.023698,0.029444,-0.263768,0.458183,0.523885,-0.245705,0.489319,0.006338,-0.059121,0.049614,0.125419,0.279885,-0.011269
waterfront,0.001544,-0.006582,0.063744,0.103818,0.021604,0.023698,1.000000,0.401857,0.016653,0.082775,0.072075,0.080588,-0.026161,0.092885,0.030285,-0.014274,-0.041910,0.086463,0.030703
view,-0.001837,0.079532,0.187737,0.284611,0.074710,0.029444,0.401857,1.000000,0.045990,0.251321,0.167649,0.276947,-0.053440,0.103917,0.084827,0.006157,-0.078400,0.280439,0.072575
condition,-0.050755,0.028472,-0.124982,-0.058753,-0.008958,-0.263768,0.016653,0.045990,1.000000,-0.144674,-0.158214,0.174105,-0.361417,-0.060618,0.003026,-0.014941,-0.106500,-0.092824,-0.003406
grade,-0.040040,0.356967,0.664983,0.762704,0.113621,0.458183,0.082775,0.251321,-0.144674,1.000000,0.755923,0.168392,0.446963,0.014414,-0.184862,0.114084,0.198372,0.713202,0.119248


In [11]:
df.columns

Index(['date', 'price', 'bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot',
       'floors', 'waterfront', 'view', 'condition', 'grade', 'sqft_above',
       'sqft_basement', 'yr_built', 'yr_renovated', 'zipcode', 'lat', 'long',
       'sqft_living15', 'sqft_lot15'],
      dtype='object')

In [12]:
cor_matrix = df.loc[:, df.columns != 'price'].corr().abs()
upper_tri = cor_matrix.where(np.triu(np.ones(cor_matrix.shape),k=1).astype(np.bool))
to_drop = [column for column in upper_tri.columns if any(upper_tri[column] > 0.8)]
print(to_drop)

['sqft_above']


In [13]:
df1 = df.drop(columns = to_drop, axis=1)

In [14]:
df1 = df

In [15]:
zipcode = pd.get_dummies(df.zipcode, prefix='zipcode')
df1 = pd.concat([df1,zipcode], axis = 1)
del df1['zipcode']

In [16]:
X = df1.loc[:, df1.columns != 'price']
y = df1['price']

In [17]:
X

,date,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,...,zipcode_98146,zipcode_98148,zipcode_98155,zipcode_98166,zipcode_98168,zipcode_98177,zipcode_98178,zipcode_98188,zipcode_98198,zipcode_98199
0,164,3,1.00,1180,5650,1.0,0,0,3,7,...,0,0,0,0,0,0,1,0,0,0
1,220,3,2.25,2570,7242,2.0,0,0,3,7,...,0,0,0,0,0,0,0,0,0,0
2,290,2,1.00,770,10000,1.0,0,0,3,6,...,0,0,0,0,0,0,0,0,0,0
3,220,4,3.00,1960,5000,1.0,0,0,5,7,...,0,0,0,0,0,0,0,0,0,0
4,283,3,2.00,1680,8080,1.0,0,0,3,8,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21608,19,3,2.50,1530,1131,3.0,0,0,3,8,...,0,0,0,0,0,0,0,0,0,0
21609,288,4,2.50,2310,5813,2.0,0,0,3,8,...,1,0,0,0,0,0,0,0,0,0
21610,52,2,0.75,1020,1350,2.0,0,0,3,7,...,0,0,0,0,0,0,0,0,0,0
21611,252,3,2.50,1600,2388,2.0,0,0,3,8,...,0,0,0,0,0,0,0,0,0,0


# Train-Test split, Scaling

In [18]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1)
X_train_cols = X_train.columns

In [19]:
scaler = StandardScaler()
scaler.fit(X_train,y_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

# Building the model

In [20]:
ridge = Ridge()
ridge_fit = ridge.fit(X_train,y_train)

In [21]:
y_pred = ridge_fit.predict(X_test)
print(np.sqrt(mean_squared_error(y_test,y_pred)))
print(np.sqrt(mean_absolute_error(y_test,y_pred)))

180407.3488280113
313.65635155179393


In [22]:
r2_score(y_test, y_pred)

0.791460596473156

## Finding the best alpha using RidgeCV

In [23]:
ridge_cv_model = RidgeCV(alphas = np.random.randint(1,1000,100), cv = 10).fit(X_train,y_train)

In [24]:
ridge_cv_model.alpha_

139

In [25]:
ridge_tuned = Ridge(alpha = ridge_cv_model.alpha_).fit(X_train,y_train)

y_pred_tuned = ridge_tuned.predict(X_test)

rmse_test = (np.sqrt(mean_squared_error(y_test,y_pred_tuned)))
mae_test = (np.sqrt(mean_absolute_error(y_test,y_pred_tuned)))
r2_test = (r2_score(y_test, y_pred_tuned))

In [26]:
#Test
result_table_test = pd.DataFrame(columns = ['Metric', 'Value'])
result_table_test.loc[0] = ['RMSE', rmse_test]
result_table_test.loc[1] = ['MAE', mae_test]
result_table_test.loc[2] = ['R2', r2_test]

print('Ridge Test metrics')
result_table_test

Ridge Test metrics


,Metric,Value
0,RMSE,180618.561321
1,MAE,313.161986
2,R2,0.790972


In [27]:
y_pred_train_tuned = ridge_tuned.predict(X_train)

rmse_train = (np.sqrt(mean_squared_error(y_train,y_pred_train_tuned)))
mae_train = (np.sqrt(mean_absolute_error(y_train,y_pred_train_tuned)))
r2_train = (r2_score(y_train, y_pred_train_tuned))

In [28]:
#Train
result_table_train = pd.DataFrame(columns = ['Metric', 'Value'])
result_table_train.loc[0] = ['RMSE', rmse_train]
result_table_train.loc[1] = ['MAE', mae_train]
result_table_train.loc[2] = ['R2', r2_train]

print('Ridge Train metrics')
result_table_train

Ridge Train metrics


,Metric,Value
0,RMSE,151256.521205
1,MAE,303.649281
2,R2,0.817891


## Analysing model coefficients

In [29]:
coef_df = pd.DataFrame(columns = ['columns', 'Ridge_coef'])
coef_df['columns'] = X_train_cols
coef_df['Ridge_coef'] = ridge_tuned.coef_
coef_df

,columns,Ridge_coef
0,date,12436.422521
1,bedrooms,-21156.071721
2,bathrooms,15938.075486
3,sqft_living,83735.909692
4,sqft_lot,8151.913165
...,...,...
83,zipcode_98177,-3017.917630
84,zipcode_98178,-14527.960749
85,zipcode_98188,-10127.754516
86,zipcode_98198,-16732.691541


In [30]:
coef_df[coef_df['columns'].isin(['sqft_above','sqft_living'])]

,columns,Ridge_coef
3,sqft_living,83735.909692
10,sqft_above,84944.243422


In [31]:
coef_df[(coef_df['Ridge_coef']<50) & (coef_df['Ridge_coef']>-50)]

,columns,Ridge_coef


In [32]:
coef_df[(coef_df['Ridge_coef']<0)]

,columns,Ridge_coef
1,bedrooms,-21156.071721
5,floors,-20222.767761
12,yr_built,-21431.588613
15,long,-17938.682435
17,sqft_lot15,-1663.415153
18,zipcode_98001,-15969.103784
19,zipcode_98002,-8733.140400
20,zipcode_98003,-16345.382681
26,zipcode_98010,-1109.786727
27,zipcode_98011,-10707.253334


# Cross Validation

In [33]:
k_folds = KFold(n_splits = 10)

scores = cross_val_score(ridge_tuned, X, y, cv = k_folds, scoring = 'r2')

print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())

Cross Validation Scores:  [0.75366342 0.76178453 0.76794843 0.75197049 0.76019774 0.78911627
 0.79068532 0.79661746 0.77320446 0.76296432]
Average CV Score:  0.7708152441775755
